In [2]:
import pickle
import numpy as np
import scipy.stats
from EM import EM
from plot import plot

In [4]:
signal_name = 'creatinine'

In [6]:
data = np.load(open('../Data/'+signal_name+'_preprocessed_data.npz', 'rb'), encoding='latin1')

In [8]:
data['y_mtx'].shape

(2434, 65)

In [16]:
def get_data(patient):
    y = data['y_mtx'][patient, :][np.newaxis, :]
    X = data['X_mtx'][patient, :, :][np.newaxis, :, :]
    c = data['c_mtx'][patient, :][np.newaxis, :]
    X_prev = data['X_prev_mtx'][patient, :, :][np.newaxis, :, :]
    return (y, X, c, X_prev)

In [18]:
# em of each individual
idv_em = []

In [21]:
# patient is the patient index
def em_individual(patient):
    y, X, c, X_prev = get_data(patient)
    em = EM(y, X, c, J=2, K=0, train_pct=.8, X_prev_given=True, X_prev=X_prev)
    em.run_EM(1)
    idv_em.append(em)

In [ ]:
%%time
%%capture
for patient in range(data['y_mtx'].shape[0]):
    em_individual(patient)

In [12]:
sum(mse)/len(mse)

0.7708313333085737

### Results Analysis
* Single vs. multi effects
    * When using single effect, prediction trajectory of individual level model appears flatter, perhaps contributing to a lower mse
* In EM for individual level model, the lowest MSE appears in different iterations for different individuals, but most often it appears after the first iteration. The total MSE increases with more iterations starting from iteration 1 (?!) 
    * This also happens sometimes in the simulation when run with only one sample. The plot seems to suggest that more iterations sometimes lead to spikes in prediction that doesn't correspond to the actual trajectory (the coefficients don't match up either), thus increasing MSE
    * This happens more drastically when we have more missingness (in the simulation)
    * Could simply because the model is learning bad coefficients, so as iterations continues it just gets worse. since with more missingness, we have less equations in the linear system so the solution is worse (?) 
* For population level, the MSE after the first iteration is also pretty close to the best MSE. In the run that produces the result, MSE rises after iterations begin then starts decreasing pretty soon
    * But at least in the simulation, MSE generally decreases with iterations or fluctuates around the lowest value